In [ ]:
#Imports
import folium
import numpy as np
import matplotlib.pyplot as plt


from eodag import EODataAccessGateway
from eodag import setup_logging

from rasterio.crs import CRS
from rioxarray.merge import merge_arrays

import eotools.shortcut as eoshort
import eotools.loading as eoload
import eotools.contrast as eocontrast


# Setup Verbose Values:
# 0: no logging and no progress bar
# 1: no logging but progress bars displayed
# 2: log at the INFO level
# 3: log at the DEBUG level (even more information)

setup_logging(verbose=1)


# EODAG - merging

EODAG (Earth Observation Data Access Gateway) is a command line tool and a Python package for searching and downloading remotely sensed images while offering a unified API for data access regardless of the data provider.

EODAG gives you an easy way to access products from more than 10 providers, with more than 50 different product types (Sentinel 1, Sentinel 2, Sentinel 3, Landsat, etc.) that can be searched and downloaded.

## Data Acquisition
The Lines of Code in this chapter, which essentialy load the data, are not further explained here, since the Notebooks 01 and 02 do that.

In [ ]:
# Get Credentials from .env file and make dirs from paths.yml
# These functions are just shortcuts from a python script and just help to keep the notebooks short and simple
workspace = eoshort.read_paths(filepath='paths.yml')
dag = EODataAccessGateway()
dag.set_preferred_provider("cop_dataspace")
deserialized_search_results = eoshort.deserialize(filename="search_results.geojson", workspace=workspace, dag=dag, log=True)

# Download multiple Products
products = deserialized_search_results
paths = dag.download_all(products)

In [ ]:
# Set Boundingbox for Area inside the Tile.
latmin, latmax = 48.1, 48.35
lonmin, lonmax = 16.1, 16.6
extent = {'lonmin': lonmin, 'latmin': latmin, 'lonmax': lonmax, 'latmax': latmax}

# Folium Map
fmap = folium.Map(location=(np.array([latmin, latmax]).mean(), np.array([lonmin, lonmax]).mean()), zoom_start=9)
folium.Rectangle(bounds=[[latmin, lonmin],[latmax, lonmax]], color="red").add_to(fmap)
folium.GeoJson(
    data=products[:],  # SearchResult has a __geo_interface__ interface used by folium to get its GeoJSON representation, single results dont work (this [2:3] instead of [2])
    tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)
folium.Marker(location=[48.18, 16.11]).add_to(fmap)
fmap

In [ ]:
# Setting common Parameters for all further image processing
common_params = dict(
    crs=CRS.from_epsg(4326),               # the downloaded images are in 4326, don't reproject them
    resolution=0.0006,                     # but lower their resolution (0.0006 should be 60m in 100km)
    extent=(lonmin,latmin,lonmax,latmax)   # and zoom over/crop the area of interest
)

# Define basic Functions for future operations
def normalized_difference(a, b):
    return (a - b*1.)/(a + b)

## Post Process - Merging

#### Merging of single Arrays

In this example two geographicaly overlapping images are merged, and the NDVI is calculated. Note that only Dataarrays are merged but not Datasets.

In [ ]:
# Sort Products 
products = sorted([p for p in deserialized_search_results], key=lambda p: p.properties["title"].split("_")[2])
products[0:2]

In [ ]:
# Load Data of Bands 4 and 8. The Images are from the same time, but different tiles.
RED_1 = eoload.get_data_regex(product=products[0], band='B04', **common_params)
RED_2 = eoload.get_data_regex(product=products[1], band='B04', **common_params)

NIR_1 = eoload.get_data_regex(product=products[0], band='B08', **common_params)
NIR_2 = eoload.get_data_regex(product=products[1], band='B08', **common_params)

# Old method for loading data
# RED_1 = products[0].get_data(band="B04", **common_params)
# RED_2 = products[1].get_data(band="B04", **common_params)

# NIR_1 = products[0].get_data(band="B08", **common_params)
# NIR_2 = products[1].get_data(band="B08", **common_params)

In [ ]:
# Using merge_arrays returns the geospatially merged data.
merged_red = merge_arrays([RED_1, RED_2], method='max') # methods = ['first', 'last', 'min', 'max', 'sum', 'count']
merged_nir = merge_arrays([NIR_1, NIR_2], method='max')

merged_ndvi = normalized_difference(merged_nir, merged_red)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11, 4))
merged_ndvi.plot(cmap='RdYlGn', center=False, ax=ax[0])
eocontrast.histogram(merged_ndvi, title="NDVI Histogram", xlim=(-1, 1), ax=ax[1])
plt.show()